# Environment Setup

In [1]:
# General
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchinfo import summary
from torchmetrics import MetricCollection, Accuracy, JaccardIndex

# CubiCasa
from floortrans.loaders import FloorplanSVG
from floortrans.loaders.augmentations import (RandomCropToSizeTorch,
                                              ResizePaddedTorch,
                                              Compose,
                                              DictToTensor,
                                              ColorJitterTorch,
                                              RandomRotations)

# Model
from model.deeplabv3plus import DeepLabV3Plus

# Release GPU memory
torch.cuda.empty_cache()
print('GPU memory has been released.')

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using device: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('Using device: CPU')


print('Setup complete.')

GPU memory has been released.
Using device: NVIDIA GeForce RTX 3060 Laptop GPU
Setup complete.


# Data Preprocessing and Augmentations

In [2]:
IMAGE_SIZE = (256, 256)

train_aug = Compose([
    # transforms.RandomChoice([
    #     RandomCropToSizeTorch(data_format='dict', size=IMAGE_SIZE),
    #     ResizePaddedTorch((0, 0), data_format='dict', size=IMAGE_SIZE)
    # ]),
    ResizePaddedTorch((0, 0), data_format='dict', size=IMAGE_SIZE),
    RandomRotations(format='cubi'),
    DictToTensor(),
    ColorJitterTorch(b_var=0.2, c_var=0.2, s_var=0.2)
])

val_aug = Compose([
    ResizePaddedTorch((0, 0), data_format='dict', size=IMAGE_SIZE),
    DictToTensor()
])

# Dataset

In [3]:
DATA_PATH = 'data/cubicasa5k/'
TRAIN_PATH = 'train.txt'
VAL_PATH = 'val.txt'
FORMAT = 'lmdb'

train_set = FloorplanSVG(
    DATA_PATH, 
    TRAIN_PATH, 
    format=FORMAT, 
    augmentations=train_aug
)

# Reduce training set for faster training (temporary)
# train_set = Subset(full_train_set, list(range(1600)))

val_set = FloorplanSVG(
    DATA_PATH, 
    VAL_PATH, 
    format=FORMAT, 
    augmentations=val_aug
)

print('Train set size:', len(train_set))
print('Validation set size:', len(val_set))

Train set size: 4200
Validation set size: 400


In [ ]:
sample = train_set[10]

print('Image shape:', sample['image'].shape)
print('Label shape (overall):', sample['label'].shape)

# Room and icon segmentation maps
print('\nLabel shape (rooms): ', sample['label'][21].shape)
print('Label shape (icons):', sample['label'][22].shape)
print()

# Heatmaps
for i in range(21):
    print(f'Label shape (heatmap {i+1}):', sample['label'][i].shape)

In [ ]:
print('Image: ', sample['image'])

In [ ]:
print('Label: ', sample['label'])

# Visualize Images and Labels

In [ ]:
# Convert back to [0, 255] range
tensor_image = sample['image'] * 255.0

np_image = tensor_image.numpy().astype(np.uint8)  # Convert to unsigned 8-bit integer

# Transpose to [H, W, 3] from [3, H, W]
np_image = np.transpose(np_image, (1, 2, 0))

# Create figure with 1 row and 3 columns for the first set of visualizations
fig, axes = plt.subplots(1, 3, figsize=(10, 3))

# Input image
axes[0].imshow(np_image)
axes[0].axis('off')  # Remove the axes
axes[0].set_title('Input Image')

# Room segmentation map
axes[1].imshow(sample['label'][21])
axes[1].axis('off')  # Remove the axes
axes[1].set_title('Room Labels')

# Icon segmentation map
axes[2].imshow(sample['label'][22])
axes[2].axis('off')
axes[2].set_title('Icon Labels')

plt.tight_layout()
plt.show()

# Now display the heatmaps, 3 heatmaps per row
num_heatmaps = 21
cols = 3
rows = (num_heatmaps + cols - 1) // cols  # Calculate number of rows needed

fig, axes = plt.subplots(rows, cols, figsize=(10, 3 * rows))

for i in range(num_heatmaps):
    row = i // cols
    col = i % cols
    axes[row, col].imshow(sample['label'][i])
    axes[row, col].axis('off')
    axes[row, col].set_title(f'Heatmap {i+1}')

# If there are empty spaces in the last row (if heatmaps < total subplots), hide them
for j in range(i + 1, rows * cols):
    fig.delaxes(axes[j // cols, j % cols])  # Remove unused subplots

plt.tight_layout()
plt.show()


# DataLoader

In [4]:
NUM_WORKERS = 0
BATCH_SIZE = 64

train_loader = DataLoader(
    train_set, 
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS, 
    shuffle=True, 
    pin_memory=True
)

val_loader = DataLoader(
    val_set, 
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS, 
    pin_memory=True
)

print(f'Length of train dataloader: {len(train_loader)} batches of size {BATCH_SIZE}')
print(f'Length of val dataloader: {len(val_loader)} batches of size {BATCH_SIZE}')

batch_sample = next(iter(train_loader))
print('\nBatch image shape: ', batch_sample['image'].shape)
print('Batch label shape: ', batch_sample['label'].shape)

Length of train dataloader: 66 batches of size 64
Length of val dataloader: 7 batches of size 64

Batch image shape:  torch.Size([64, 3, 256, 256])
Batch label shape:  torch.Size([64, 23, 256, 256])


# Model Setup

For reference, here are the 23 classes:  

- **Rooms (12):** "Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"  

- **Icons (11):** "No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"

In [6]:
model = DeepLabV3Plus(backbone='mobilenetv2', attention=True)
model.to(device)

summary(model, input_size=(32, 3, 512, 512))

Layer (type:depth-idx)                                       Output Shape              Param #
DeepLabV3Plus                                                [32, 12, 512, 512]        --
├─Backbone: 1-1                                              [32, 24, 128, 128]        --
│    └─MobileNetV2: 2-1                                      --                        1,281,000
│    │    └─Sequential: 3-1                                  --                        2,223,872
├─ASPP: 1-2                                                  [32, 256, 16, 16]         --
│    └─Conv2d: 2-2                                           [32, 256, 16, 16]         327,680
│    └─AtrousConv: 2-3                                       [32, 256, 16, 16]         --
│    │    └─DepthwiseSeparableConv: 3-2                      [32, 256, 16, 16]         339,712
│    │    └─ChannelAttention: 3-3                            [32, 256, 16, 16]         8,464
│    └─AtrousConv: 2-4                                       [32, 25

In [10]:
# # Verifying output shapes
# sample_input = torch.randn(BATCH_SIZE, 3, IMAGE_SIZE[0], IMAGE_SIZE[1]).to(device)

# room_output, icon_output, heatmap_output = model(sample_input)
# print('Room Output Shape:', room_output.shape)  # Expected: [16, 12, 256, 256]
# print('Icon Output Shape:', icon_output.shape)  # Expected: [16, 11, 256, 256]
# print('Heatmap Output Shape:', heatmap_output.shape)  # Expected: [16, 21, 256, 256]

# Loss Function: Multi-Task Uncertainty Loss 

In [10]:
class MultiTaskUncertaintyLoss(nn.Module):
    def __init__(self, num_tasks=2):
        super().__init__()

        # Learnable parameter for each task
        self.log_sigma = nn.Parameter(torch.zeros(num_tasks))


    def forward(self, losses):
        """
        Forward pass to compute the total multi-task loss.
        Args:
            losses (list of tensors): List of individual task losses.
        Returns:
            total_loss (tensor): The final weighted multi-task loss.
        """

        # Tensor to avoid issues with autograd
        total_loss = torch.tensor(0.0, requires_grad=True).to(losses[0].device)

        for i, loss in enumerate(losses):
            precision_weight = torch.exp(-self.log_sigma[i])
            task_loss = precision_weight * loss + self.log_sigma[i]
            total_loss = total_loss + task_loss
        
        return total_loss

# Updated Training and Validation Loop

In [ ]:
# Individual loss functions for each task
room_criterion = nn.CrossEntropyLoss()
icon_criterion = nn.CrossEntropyLoss()
heatmap_criterion = nn.MSELoss()

# Multi-task uncertainty loss
multitask_criterion = MultiTaskUncertaintyLoss(num_tasks=3).to(device)

# Based on CubiCasa5k training setup
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-8)

# Reduce learning rate if validation loss doesnt improve for 20 epochs
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20)

# Evaluation metrics
room_metrics = MetricCollection({
    'mpa': Accuracy(task='multiclass', num_classes=12, average='macro'),
    'miou': JaccardIndex(task='multiclass', num_classes=12, average='macro'),
    'fwiou': JaccardIndex(task='multiclass', num_classes=12, average='weighted'),
}).to(device)

icon_metrics = MetricCollection({
    'mpa': Accuracy(task='multiclass', num_classes=11, average='macro'),
    'miou': JaccardIndex(task='multiclass', num_classes=11, average='macro'),
    'fwiou': JaccardIndex(task='multiclass', num_classes=11, average='weighted'),
}).to(device)


# Training loop
def train(model, dataloader, optimizer, device):
    model.train()

    total_loss = 0.0

    # To show progress bar
    for batch in tqdm(dataloader, desc='Training'):
        images = batch['image'].float().to(device)
        room_labels = batch['label'][:, 21].long().to(device)
        icon_labels = batch['label'][:, 22].long().to(device)
        heatmap_labels = batch['label'][:, 0:21].to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        room_output, icon_output, heatmap_output = model(images)

        # Compute individual losses per task
        room_loss = room_criterion(room_output, room_labels)
        icon_loss = icon_criterion(icon_output, icon_labels)
        heatmap_loss = heatmap_criterion(heatmap_output, heatmap_labels)

        # Compute combined loss
        combined_loss = multitask_criterion([room_loss, icon_loss, heatmap_loss])

        # Backward pass and optimization
        combined_loss.backward()
        optimizer.step()

        # Accumulate the scalar value of the loss
        total_loss += combined_loss.item()

        # Get model predictions
        room_preds = torch.argmax(room_output, dim=1)
        icon_preds = torch.argmax(icon_output, dim=1)

        # Update metrics
        room_metrics.update(room_preds, room_labels)
        icon_metrics.update(icon_preds, icon_labels)

    # Compute epoch loss and metrics
    epoch_loss = total_loss / len(dataloader)
    epoch_room_metrics = room_metrics.compute()
    epoch_icon_metrics = icon_metrics.compute()

    # Reset metrics
    room_metrics.reset()
    icon_metrics.reset()

    return epoch_loss, epoch_room_metrics, epoch_icon_metrics


# Validation loop
def validate(model, dataloader, device):
    model.eval()

    total_loss = 0.0

    # Don't compute gradients
    with torch.no_grad():

        # Wrap dataloader with tqdm to show progress bar
        for batch in tqdm(dataloader, desc='Validating'):
            images = batch['image'].float().to(device)

            room_labels = batch['label'][:, 21].long().to(device)
            icon_labels = batch['label'][:, 22].long().to(device)
            heatmap_labels = batch['label'][:, 0:21].to(device)

            # Forward pass
            room_output, icon_output, heatmap_output = model(images)

            # Compute individual losses
            room_loss = room_criterion(room_output, room_labels)
            icon_loss = icon_criterion(icon_output, icon_labels)
            heatmap_loss = heatmap_criterion(heatmap_output, heatmap_labels)

            # Compute combined loss
            combined_loss = multitask_criterion([room_loss, icon_loss, heatmap_loss])

            # Accumulate the scalar value of the loss
            total_loss += combined_loss.item()

            # Get model predictions
            room_preds = torch.argmax(room_output, dim=1)
            icon_preds = torch.argmax(icon_output, dim=1)

            # Update metrics
            room_metrics.update(room_preds, room_labels)
            icon_metrics.update(icon_preds, icon_labels)
    
    # Compute epoch loss and metrics
    epoch_loss = total_loss / len(dataloader)
    epoch_room_metrics = room_metrics.compute()
    epoch_icon_metrics = icon_metrics.compute()

    # Reset metrics
    room_metrics.reset()
    icon_metrics.reset()
    
    return epoch_loss, epoch_room_metrics, epoch_icon_metrics


# Train and validate model
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"EPOCH {epoch+1}/{num_epochs}")

    train_loss, train_room_metrics, train_icon_metrics = train(model, train_loader, optimizer, device)
    val_loss, val_room_metrics, val_icon_metrics = validate(model, val_loader, device)

    print('\nTRAIN RESULTS')
    print(f'Loss: {train_loss:.4f}')
    print(f'Rooms: {train_room_metrics}')
    print(f'Icons: {train_icon_metrics}')

    print('\nVALIDATION RESULTS')
    print(f'Loss: {val_loss:.4f}')
    print(f'Rooms: {val_room_metrics}')
    print(f'Icons: {val_icon_metrics}')

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'saved_models/deeplabv3plus_{model.backbone_name}.pt')
        print(f'\nBest model saved with validation loss: {best_val_loss:.4f}')
    else:
        print(f'\nValidation loss did not improve.Best loss is still: {best_val_loss:.4f}')

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')

# Training and Validation Loop

In [14]:
# EPOCHS = 400
# OPTIMIZER = torch.optim.Adam(model.parameters(), lr=0.0001)
# CRITERION = MultiTaskLossWrapper(task_num=2)

# def timer(start_time = None): 
#     return time.time() if start_time == None else time.time() - start_time


# def train_evaluate(model, 
#                    train_loader, 
#                    val_loader, 
#                    device,
#                    loss_fn,
#                    optimizer, 
#                    epochs,
#                    early_stop_threshold=15,
#                    save_prefix='deeplabv3plus',
#                    save_path='saved_models'):
    
#     # Store results, to be returned
#     train_loss_list = []
#     train_room_cpa_list = []
#     train_room_mpa_list = []
#     train_room_miou_list = []
#     train_room_fwiou_list = []
#     train_icon_cpa_list = []
#     train_icon_mpa_list = []
#     train_icon_miou_list = []
#     train_icon_fwiou_list = []

#     val_loss_list = []
#     val_room_cpa_list = []
#     val_room_mpa_list = []
#     val_room_miou_list = []
#     val_room_fwiou_list = []
#     val_icon_cpa_list = []
#     val_icon_mpa_list = []
#     val_icon_miou_list = []
#     val_icon_fwiou_list = []

#     # Training metrics
#     train_room_cpa = Accuracy(task='multiclass', num_classes=12, average=None).to(device)
#     train_room_mpa = Accuracy(task='multiclass', num_classes=12, average='macro').to(device)
#     train_room_miou = JaccardIndex(task='multiclass', num_classes=12, average='macro').to(device)
#     train_room_fwiou = JaccardIndex(task='multiclass', num_classes=12, average='weighted').to(device)

#     train_icon_cpa = Accuracy(task='multiclass', num_classes=11, average=None).to(device)
#     train_icon_mpa = Accuracy(task='multiclass', num_classes=11, average='macro').to(device)
#     train_icon_miou = JaccardIndex(task='multiclass', num_classes=11, average='macro').to(device)
#     train_icon_fwiou = JaccardIndex(task='multiclass', num_classes=11, average='weighted').to(device)

#     # Validation metrics
#     val_room_cpa = Accuracy(task='multiclass', num_classes=12, average=None).to(device)
#     val_room_mpa = Accuracy(task='multiclass', num_classes=12, average='macro').to(device)
#     val_room_miou = JaccardIndex(task='multiclass', num_classes=12, average='macro').to(device)
#     val_room_fwiou = JaccardIndex(task='multiclass', num_classes=12, average='weighted').to(device)

#     val_icon_cpa = Accuracy(task='multiclass', num_classes=11, average=None).to(device)
#     val_icon_mpa = Accuracy(task='multiclass', num_classes=11, average='macro').to(device)
#     val_icon_miou = JaccardIndex(task='multiclass', num_classes=11, average='macro').to(device)
#     val_icon_fwiou = JaccardIndex(task='multiclass', num_classes=11, average='weighted').to(device)
    
#     best_loss = np.inf
#     not_improving = 0
    
#     # Save models in this directory
#     os.makedirs(save_path, exist_ok=True)

#     # Start timer
#     train_start = timer()
#     print('Start training process...')

#     for epoch in range(1, epochs + 1):
#         epoch_start = timer()
        
#         # Training loop
#         print(f'Epoch {epoch} train process started...')
#         model.train()

#         epoch_train_loss = 0.0

#         for batch in tqdm(train_loader):
#             images = batch['image'].to(device)
#             room_labels = batch['label'][:, 0].to(device)
#             icon_labels = batch['label'][:, 1].to(device)

#             # Reset gradients since PyTorch accumulates previous gradients
#             optimizer.zero_grad()

#             # Forward pass
#             room_output, icon_output = model(images)

#             # Calculate loss
#             loss = loss_fn(room_output, icon_output, room_labels, icon_labels)
#             epoch_train_loss += loss.item()

#             # Backward pass
#             loss.backward()

#             # Update weights
#             optimizer.step()

#             # Update metrics
#             train_room_cpa.update(room_output, room_labels)
#             train_room_mpa.update(room_output, room_labels)
#             train_room_miou.update(room_output, room_labels)
#             train_room_fwiou.update(room_output, room_labels)

#             train_icon_cpa.update(icon_output, icon_labels)
#             train_icon_mpa.update(icon_output, icon_labels)
#             train_icon_miou.update(icon_output, icon_labels)
#             train_icon_fwiou.update(icon_output, icon_labels)

#         # Calculate training metrics
#         train_room_cpa_value = train_room_cpa.compute()
#         train_room_mpa_value = train_room_mpa.compute().item()
#         train_room_miou_value = train_room_miou.compute().item()
#         train_room_fwiou_value = train_room_fwiou.compute().item()

#         train_icon_cpa_value = train_icon_cpa.compute()
#         train_icon_mpa_value = train_icon_mpa.compute().item()
#         train_icon_miou_value = train_icon_miou.compute().item()
#         train_icon_fwiou_value = train_icon_fwiou.compute().item()

#         # Reset metrics
#         train_room_cpa.reset()
#         train_room_mpa.reset()
#         train_room_miou.reset()
#         train_room_fwiou.reset()

#         train_icon_cpa.reset()
#         train_icon_mpa.reset()
#         train_icon_miou.reset()
#         train_icon_fwiou.reset()

        
#         # Validation loop
#         print(f'Epoch {epoch} validation process started...')
#         model.eval()

#         epoch_val_loss = 0.0

#         with torch.no_grad():
#             for batch in tqdm(val_loader):
#                 images = batch['image'].to(device)
#                 room_labels = batch['label'][:, 0].to(device)
#                 icon_labels = batch['label'][:, 1].to(device)

#                 # Get model predictions
#                 room_output, icon_output = model(images)

#                 # Calculate loss
#                 loss = loss_fn(room_output, icon_output, room_labels, icon_labels)
#                 epoch_val_loss += loss.item()

#                 # Update metrics
#                 val_room_cpa.update(room_output, room_labels)
#                 val_room_mpa.update(room_output, room_labels)
#                 val_room_miou.update(room_output, room_labels)
#                 val_room_fwiou.update(room_output, room_labels)

#                 val_icon_cpa.update(icon_output, icon_labels)
#                 val_icon_mpa.update(icon_output, icon_labels)
#                 val_icon_miou.update(icon_output, icon_labels)
#                 val_icon_fwiou.update(icon_output, icon_labels)

#         # Calculate validation metrics
#         val_room_cpa_value = val_room_cpa.compute()
#         val_room_mpa_value = val_room_mpa.compute().item()
#         val_room_miou_value = val_room_miou.compute().item()
#         val_room_fwiou_value = val_room_fwiou.compute().item()

#         val_icon_cpa_value = val_icon_cpa.compute()
#         val_icon_mpa_value = val_icon_mpa.compute().item()
#         val_icon_miou_value = val_icon_miou.compute().item()
#         val_icon_fwiou_value = val_icon_fwiou.compute().item()

#         # Reset metrics
#         val_room_cpa.reset()
#         val_room_mpa.reset()
#         val_room_miou.reset()
#         val_room_fwiou.reset()

#         val_icon_cpa.reset()
#         val_icon_mpa.reset()
#         val_icon_miou.reset()
#         val_icon_fwiou.reset()


#         # Print epoch results
#         print(f'Epoch {epoch} train process is completed.')
#         print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
#         print(f'\nEpoch {epoch} train process results:\n')

#         print(f'Train Time: {timer(epoch_start):.3f} secs')

#         # Combine room and icon metrics for training, including loss
#         print(f'Train Room - Loss: {epoch_train_loss / len(train_loader):.3f}, MPA: {train_room_mpa_value:.3f}, mIOU: {train_room_miou_value:.3f}, fwIOU: {train_room_fwiou_value:.3f}')
#         print(f'Train Icon - Loss: {epoch_train_loss / len(train_loader):.3f}, MPA: {train_icon_mpa_value:.3f}, mIOU: {train_icon_miou_value:.3f}, fwIOU: {train_icon_fwiou_value:.3f}')

#         print(f'\nVal process results:')

#         # Combine room and icon metrics for validation, including loss
#         print(f'Val Room - Loss: {epoch_val_loss / len(val_loader):.3f}, MPA: {val_room_mpa_value:.3f}, mIOU: {val_room_miou_value:.3f}, fwIOU: {val_room_fwiou_value:.3f}')
#         print(f'Val Icon - Loss: {epoch_val_loss / len(val_loader):.3f}, MPA: {val_icon_mpa_value:.3f}, mIOU: {val_icon_miou_value:.3f}, fwIOU: {val_icon_fwiou_value:.3f}')


#         # Append results
#         train_loss_list.append(epoch_train_loss / len(train_loader))
#         train_room_cpa_list.append(train_room_cpa_value)
#         train_room_mpa_list.append(train_room_mpa_value)
#         train_room_miou_list.append(train_room_miou_value)
#         train_room_fwiou_list.append(train_room_fwiou_value)
#         train_icon_cpa_list.append(train_icon_cpa_value)
#         train_icon_mpa_list.append(train_icon_mpa_value)
#         train_icon_miou_list.append(train_icon_miou_value)
#         train_icon_fwiou_list.append(train_icon_fwiou_value)

#         val_loss_list.append(epoch_val_loss / len(val_loader))
#         val_room_cpa_list.append(val_room_cpa_value)
#         val_room_mpa_list.append(val_room_mpa_value)
#         val_room_miou_list.append(val_room_miou_value)
#         val_room_fwiou_list.append(val_room_fwiou_value)
#         val_icon_cpa_list.append(val_icon_cpa_value)
#         val_icon_mpa_list.append(val_icon_mpa_value)
#         val_icon_miou_list.append(val_icon_miou_value)
#         val_icon_fwiou_list.append(val_icon_fwiou_value)

#         # Save model if validation loss is improved
#         if (epoch_val_loss / len(val_loader)) < best_loss:
#             print(f'\nLoss decreased from {best_loss:.3f} to {(epoch_val_loss / len(val_loader)):.3f}!')
#             best_loss = (epoch_val_loss / len(val_loader))

#             not_improving = 0 # Reset counter

#             print('Saving the model with the best loss value...')
#             torch.save(model.state_dict(), f'{save_path}/{save_prefix}.pt')
        
#         else:
#             not_improving += 1
#             print(f'\nLoss did not decrease for {not_improving} epoch(s)!')

#             if not_improving == early_stop_threshold:
#                 print(f'Stopping training process because loss did not decrease for {early_stop_threshold} epochs!')
#                 break
        
#         print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

#     print(f'Train process is completed in {(timer(train_start)) / 60:.3f} minutes.')


#     return {
#         'train_loss': train_loss_list,
#         'train_room_cpa': train_room_cpa_list,
#         'train_room_mpa': train_room_mpa_list,
#         'train_room_miou': train_room_miou_list,
#         'train_room_fwiou': train_room_fwiou_list,
#         'train_icon_cpa': train_icon_cpa_list,
#         'train_icon_mpa': train_icon_mpa_list,
#         'train_icon_miou': train_icon_miou_list,
#         'train_icon_fwiou': train_icon_fwiou_list,
#         'val_loss': val_loss_list,
#         'val_room_cpa': val_room_cpa_list,
#         'val_room_mpa': val_room_mpa_list,
#         'val_room_miou': val_room_miou_list,
#         'val_room_fwiou': val_room_fwiou_list,
#         'val_icon_cpa': val_icon_cpa_list,
#         'val_icon_mpa': val_icon_mpa_list,
#         'val_icon_miou': val_icon_miou_list,
#         'val_icon_fwiou': val_icon_fwiou_list
#     }


# # Run training and validation process
# history = train_evaluate(model,
#                          train_loader,
#                          val_loader,
#                          device,
#                          CRITERION,
#                          OPTIMIZER,
#                          EPOCHS,
#                          save_prefix=f'deeplabv3plus_{model.backbone_name}')

# Visualizing Performance

In [15]:
# class Plot():
#     def __init__(self, results):
#         self.results = results

#         self.visualize(metric1="tr_iou", 
#                        metric2="val_iou", 
#                        label1="Train IoU",
#                        label2 ="Validation IoU", 
#                        title="Mean Intersection Over Union Learning Curve", 
#                        ylabel="mIoU Score")

#         self.visualize(metric1="tr_pa", 
#                        metric2="val_pa", 
#                        label1="Train PA",
#                        label2="Validation PA", 
#                        title="Pixel Accuracy Learning Curve", 
#                        ylabel="PA Score")

#         self.visualize(metric1="tr_loss", 
#                        metric2="val_loss", 
#                        label1="Train Loss",
#                        label2="Validation Loss", 
#                        title="Loss Learning Curve", 
#                        ylabel="Loss Value")

#     def plot(self, metric, label): 
#         plt.plot(self.results[metric], label=label)

#     def decorate(self, ylabel, title): 
#         plt.title(title)
#         plt.xlabel("Epochs")
#         plt.ylabel(ylabel)
#         plt.legend()
#         plt.show()

#     def visualize(self, metric1, metric2, label1, label2, title, ylabel):
#         plt.figure(figsize=(10, 5))
#         self.plot(metric1, label1)
#         self.plot(metric2, label2)
#         self.decorate(ylabel, title)


# Plot(history)

# Testing 

In [16]:
# class Test():
#     def __init__(self, model, test_loader, loss_fn, device):
#         self.model = model
#         self.test_loader = test_loader
#         self.loss_fn = loss_fn
#         self.device = device
    
#     def run(self):
#         self.model.eval()
#         test_loss = 0
#         test_iou = 0
#         test_pixel_acc = 0
#         test_len = len(self.test_loader)

#         imgs = []
#         gts = []
#         preds = []

#         with torch.no_grad():
#             for batch in tqdm(self.test_loader):
#                 imgs_batch = batch['image']
#                 gts_batch = batch['label']
#                 imgs_batch, gts_batch = imgs_batch.to(self.device), gts_batch.to(self.device)

#                 # Forward pass
#                 preds_batch = self.model(imgs_batch)
                
#                 # Calculate metrics
#                 metrics = Metrics(preds_batch, gts_batch, self.loss_fn)
#                 test_loss += metrics.loss().item()
#                 test_iou += metrics.mIOU()
#                 test_pixel_acc += metrics.PixelAcc()

#                 # Collect data for visualization
#                 preds_batch = torch.argmax(preds_batch, dim=1)
#                 imgs.extend(imgs_batch.cpu())
#                 gts.extend(gts_batch.cpu())
#                 preds.extend(preds_batch.cpu())

#         # Calculate average metrics
#         test_loss /= test_len
#         test_iou /= test_len
#         test_pixel_acc /= test_len

#         return imgs, gts, preds, test_loss, test_iou, test_pixel_acc


# test = Test(model, test_loader, CRITERION, device)
# imgs, gts, preds, test_loss, test_iou, test_pixel_acc = test.run()

# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test mIoU: {test_iou:.4f}")
# print(f"Test PA: {test_pixel_acc:.4f}")